## Functions

In [1]:

import LyndonWords
import importlib
importlib.reload(LyndonWords)
from LyndonWords import *
import itertools
from tqdm.notebook import tqdm
import math
import numpy as np
def genPermutations(type,n,k):
    rootSystemPermutations = np.empty(math.factorial(n+1),dtype=object)
    v = 0
    for i in tqdm(itertools.permutations(np.array(range(n+1))),total=math.factorial(n+1)):
        rootSystemPermutations[v] = rootSystem(i,type,k)
        v+=1
    return rootSystemPermutations

In [2]:
#TODO: Updated bases on new claim on monotonicity
def checkMonotoneIncreasingImpCond(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex-1] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [3]:
def checkCondImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex-1] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [4]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [5]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [6]:
def exportByLength(rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

In [26]:
def checkDeltaPattern(perms,k=2,printDelta = False):
    for i in perms:
        baseImaginaryWords = i.getWords(i.delta)
        deltaWords = i.getWords(k*i.delta)
        flag = True
        for j in range(len(deltaWords)):
            #Tests if split is at last word
            if(deltaWords[j][-1] != baseImaginaryWords[j][-1]
               or word.letterListCmp(deltaWords[j][:i.deltaHeight-1],baseImaginaryWords[j][:i.deltaHeight-1]) != 0):
                cofacSplit = i.costfac(baseImaginaryWords[j])[0].height
                deltaWord = deltaWords[j]
                if(
                    word.letterListCmp(deltaWord[:cofacSplit], baseImaginaryWords[j][:cofacSplit]) == 0
                    and word.letterListCmp(deltaWord[cofacSplit-i.deltaHeight:], baseImaginaryWords[j][cofacSplit:]) == 0
                   ):
                    matchingFlag = True
                    baseDeltaIndex = 0
                    for baseDelta in baseImaginaryWords:
                        deltaFlag = False
                        for deltaOff in range((len(deltaWord)//i.deltaHeight) - 1):
                            if(word.letterListCmp(deltaWord[cofacSplit+(deltaOff*i.deltaHeight):cofacSplit+(deltaOff+1)*i.deltaHeight],
                                                baseDelta.string) == 0):
                                deltaFlag = True
                                break
                        matchingFlag=deltaFlag
                        if(matchingFlag):
                            break
                        baseDeltaIndex+=1
                    if(matchingFlag):
                        continue
                if(flag):
                    print(i.ordering)
                    flag = False
                if(printDelta):
                    print(f"  SL_{j}(\delta): " + '-'.join([str(i) for i in i.costfac(i.getWords(i.delta)[j])]))
                print(f"  SL_{j}({k}*\delta): " + '-'.join([str(i) for i in i.costfac((deltaWords[j]))]))

In [8]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [9]:
def compareRootSystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.getWords(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.getWords(i[j].weights)[j]))

In [10]:
def generateEquivalence(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compareRootSystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

In [11]:
exportByLength(rootSystem([1,4,5,2,3,0],'D',3),True)

## Type C

In [18]:
Cperms = genPermutations('C',5,5)

  0%|          | 0/720 [00:00<?, ?it/s]

In [21]:
print(*[f"{str(i[0])}: {i[1]}\n" for i in checkMonotoneIncreasingImpCond(Cperms)])

5<4<3<2<1<0: [0 0 0 0 0 1]
 5<4<3<2<1<0: [1 0 0 0 0 1]
 5<4<3<2<1<0: [1 1 0 0 0 1]
 5<4<3<2<1<0: [2 0 0 0 0 1]
 5<4<3<2<1<0: [2 1 0 0 0 1]
 5<4<3<2<1<0: [2 2 0 0 0 1]



In [22]:
checkCondImpMonotoneIncreasing(Cperms)

[('1<3<0<2<4<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<2<5<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<5<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<4<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<3<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<4<3<0<5<2', '1,2,0,3,4,5,4,3,2')

In [20]:
checkDeltaPattern(Cperms,2,printDelta=True)

In [25]:
checkHDeltasRepeat(Cperms)

[]

In [26]:
index=250
print(Cperms[index].ordering)
[print(*list(Cperms[index].costfac(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

2<0<3<5<1<4
2,1,3,4,0-2,1,3,4,5-[1 1 1 1 1]
2,1,3,4-2,1,3,4,5,0-[1 1 1 1 0]
2,1,3,0-2,1,3,4,5,4-[1 1 1 2 1]
2,1,0-2,1,3,4,5,4,3-[1 1 2 2 1]
2,3,4,5,4,3-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [27]:
for i in generateEquivalence(Cperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<0<4<5 

## Type G

In [143]:
G2perms = genPermutations('G',2,5)

  0%|          | 0/6 [00:00<?, ?it/s]

In [144]:
checkCondImpMonotoneIncreasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [145]:
checkMonotoneIncreasingImpCond(G2perms)

[]

In [146]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [164]:
checkDeltaPattern(G2perms)

In [148]:
checkHDeltasRepeat(G2perms)

[]

In [149]:
for i in generateEquivalence(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [21]:
Bperms = genPermutations('B',5,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [114]:
checkCondImpMonotoneIncreasing(Bperms)

[('2<0<1<3<4<5', '2,3,4,5,5,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,3,1

In [22]:
checkMonotoneIncreasingImpCond(Bperms)

[]

In [116]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [28]:
checkDeltaPattern(Bperms,printDelta=True)

4
2
SL_{5}(d)
34554SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)SL_{3}(d)SL_{3}(d)32102
3<0<1<4<2<5


In [118]:
checkHDeltasRepeat(Bperms)

[]

In [119]:
for i in generateEquivalence(Bperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<4<0<5 

## Type D

In [15]:
Dperms = genPermutations('D',5,5)

  0%|          | 0/720 [00:00<?, ?it/s]

In [130]:
checkMonotoneIncreasingImpCond(Dperms)

[('5<4<3<2<1<0', array([0, 0, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([0, 1, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([1, 1, 0, 0, 0, 1]))]

In [131]:
checkCondImpMonotoneIncreasing(Dperms)

[('2<0<4<5<1<3', '2,3,1,5,4,3'),
 ('2<0<4<5<1<3', '2,3,1,5,4,3,0'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3,0'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3,1'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3,1'),
 ('2<4<0<5<1<3', '2,3,1,5,4,3'),
 ('2<4<0<5<1<3', '2,3,1,5,0,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,1,4,3'),
 ('2<4<5<0<1<3', '2,3,1,5,4,3'),
 ('2<4<5<0<1<3', '2,3,1,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,1,5,4,3'),
 ('2<5<0<4<1<3', '2,3,1,4,5,3'),
 ('2<5<0<4<1<3', '2,3,1,4,0,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,1,5,3'),
 ('2<5<4<0<1<3', '2,3,1,4,5,3'),
 ('2<5<4<0<1<3', '2,3,1,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,1,4,5,3'),
 ('3<0<1<4<5<2', '3,2,5,1,0,2'),
 ('3<0<1<4<5<2', '3,2,5,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,5,1,0,2'),
 ('3<0<4<1<5<2', '3,2,5,1,0,2'),
 ('3<0<4<1<5<2'

In [17]:
checkDeltaPattern(Dperms,2,printDelta=True)

In [133]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [134]:
checkHDeltasRepeat(Dperms)

[]

In [135]:
for i in generateEquivalence(Dperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<2<3<4<0<5 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<4<0<5
1<2<3<5<0<4 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<5<0<4
1<2<4<3<5<0 = 1<2<4<3<0<5
1<2<5<3<4<0 = 1<2<5<3<0<4
1<3<2<4<0<5 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<4<0<5
1<3<2<5<0<4 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<5<0<4
1<3<4<2<5<0 = 1<3<4<2<0<5
1<3<5<2<4<0 = 1<3<5<2<0<4
1<4<2<3<5<0 = 1<4<2<3<0<5
1<4<3<2<5<0 = 1<4<3<2<0<5
1<5<2<3<4<0 

## Type F

In [136]:
Fperms = genPermutations('F',4,4)

  0%|          | 0/120 [00:00<?, ?it/s]

In [137]:
checkMonotoneIncreasingImpCond(Fperms)

[]

In [138]:
checkCondImpMonotoneIncreasing(Fperms)

[('1<0<2<3<4', '1,2,3,4,3,4,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,3'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,0'),
 ('1<0<3<

In [167]:
checkDeltaPattern(Fperms,2,printDelta=True)

In [140]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [141]:
checkHDeltasRepeat(Fperms)

[]

In [142]:
for i in generateEquivalence(Fperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1 = 0<2<3<1<4
0<3<2<4<1 = 0<3<2<1<4
1<2<3<0<4 = 1<2<0<3<4
1<2<3<4<0 = 1<2<0<3<4
1<2<3<4<0 = 1<2<3<0<4
1<2<4<0<3 = 1<2<0<4<3
1<2<4<3<0 = 1<2<0<4<3
1<2<4<3<0 = 1<2<4<0<3
1<3<2<4<0 = 1<3<2<0<4
1<3<4<0<2 = 1<3<0<4<2
1<4<2<3<0 = 1<4<2<0<3
2<0<3<4<1 = 2<0<3<1<4
2<1<3<0<4 = 2<1<0<3<4
2<1<3<4<0 = 2<1<0<3<4
2<1<3<4<0 = 2<1<3<0<4
2<1<4<0<3 = 2<1<0<4<3
2<1<4<3<0 = 2<1<0<4<3
2<1<4<3<0 = 2<1<4<0<3
2<3<0<4<1 = 2<3<0<1<4
2<3<1<4<0 = 2<3<1<0<4
2<3<4<0<1 = 2<3<0<1<4
2<3<4<0<1 = 2<3<0<4<1
2<3<4<1<0 = 2<3<1<0<4
2<3<4<1<0 = 2<3<1<4<0
2<4<1<3<0 = 2<4<1<0<3
3<0<2<4<1 = 3<0<2<1<4
3<1<2<4<0 = 3<1<2<0<4
3<1<4<0<2 = 3<1<0<4<2
3<2<0<4<1 = 3<2<0<1<4
3<2<1<4<0 = 3<2<1<0<4
3<2<4<0<1 = 3<2<0<1<4
3<2<4<0<1 = 3<2<0<4<1
3<2<4<1<0 = 3<2<1<0<4
3<2<4<1<0 = 3<2<1<4<0
4<1<2<3<0 = 4<1<2<0<3
4<2<1<3<0 = 4<2<1<0<3
